# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [2]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info()


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


Timelink version: 1.1.15
Project name: tutorial
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Database type: sqlite
Database name: tutorial
Kleio image: timelinkserver/kleio-server
Kleio server token: tQxiv...
Kleio server URL: http://127.0.0.1:8090
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Kleio server container: fervent_clarke
Kleio version requested: latest
Kleio server version: 12.6.577 (2024-10-24 16:53:53)
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=tutorial, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial, db_type=sqlite, db_name=tutorial, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest

TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [3]:
tlnb.table_row_count_df()

,table,count
0,acts,0
1,alembic_version,1
2,aregisters,0
3,attributes,0
4,blinks,0
5,class_attributes,63
6,classes,13
7,entities,13
8,geoentities,0
9,goods,0


### Show the kleio files available 

In [4]:
kleio_files = tlnb.get_kleio_files()
kleio_files

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,kleio/b1685.cli,b1685.cli,2024-12-06 03:39:36.260666+00:00,T,NaT,NaN,NaN,N,0,0,None,None,None,None,None
1,kleio/dehergne-a.cli,dehergne-a.cli,2024-12-06 03:39:36.262922+00:00,T,NaT,NaN,NaN,N,0,0,None,None,None,None,None
2,kleio/dehergne-locations-1644.cli,dehergne-locations-1644.cli,2024-12-06 03:39:36.263832+00:00,T,NaT,NaN,NaN,N,0,0,None,None,None,None,None
3,kleio/real-entities/real-entities.cli,real-entities.cli,2024-12-06 03:39:36.264506+00:00,T,2024-10-16 04:21:00+00:00,0.0,0.0,N,0,0,None,None,None,/rest/reports/kleio/real-entities/real-entitie...,/rest/exports/kleio/real-entities/real-entitie...


## Optionally clean translations 

In [5]:
tlnb.kleio_server.translation_clean("", recurse="yes")

['kleio/real-entities/real-entities.cli',
 'kleio/dehergne-locations-1644.cli',
 'kleio/dehergne-a.cli',
 'kleio/b1685.cli']


# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [6]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()
# get the import status
import_status = tlnb.get_import_status()
# list those with import_error > 0 or error > 0
errors = import_status.query("import_errors > 0 or errors > 0")
if len(errors) > 0:
    print(errors)

                   path            name    size directory  \
1  kleio/dehergne-a.cli  dehergne-a.cli  119167     kleio   

                          modified              modified_iso  \
1 2024-12-06 08:56:37.420016+00:00 2024-12-06 08:56:37+00:00   

       modified_string                     qtime         qtime_string  \
1  2024-12-06 08:56:37 1970-01-01 00:00:00+00:00  1970-01-01 00:00:00   

                           source_url  ...  \
1  /rest/sources/kleio/dehergne-a.cli  ...   

                                             version  \
1  KleioTranslator - server version 12.6 - build ...   

                              rpt_url                             xml_url  \
1  /rest/reports/kleio/dehergne-a.rpt  /rest/exports/kleio/dehergne-a.xml   

   import_status  import_errors import_warnings  \
1              E              3               3   

                                    import_error_rpt import_warning_rpt  \
1  ERROR: dehergne-a.cli processing same_as relat...        No

## Check the status of the files

Check the import status of the translated files

* I: Imported
* E: Imported with error
* W: Imported with warnings no errors
* N: Not imported
* U: Translation updated need to reimport

In [7]:
imported_files_df = tlnb.get_import_status()
imported_files_df[["import_status","import_errors","import_warnings","name","errors","warnings","path"]].sort_values("name")


,import_status,import_errors,import_warnings,name,errors,warnings,path
0,I,0,0,b1685.cli,0,1,kleio/b1685.cli
1,E,3,3,dehergne-a.cli,0,0,kleio/dehergne-a.cli
2,I,0,0,dehergne-locations-1644.cli,0,0,kleio/dehergne-locations-1644.cli
3,I,0,0,real-entities.cli,0,0,kleio/real-entities/real-entities.cli


### Check a translation report

If needed to check for errors.


In [8]:
file_number=1
rpt = tlnb.get_translation_report(imported_files_df, file_number)
print(rpt)


KleioTranslator - server version 12.6 - build 577 2024-10-24 16:53:53
6-12-2024 8-56

Processing data file dehergne-a.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 185
Obs: 
** Processing source fonte$dehergne-a
7: lista$dehergne-notices-a
*** End of File

Line 541 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-belchior-miguel-carneiro-leitao) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 814 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-jean-regis-lieou) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 876 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-alessandro-cicero) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.

Structure file: /usr/local/timelink/clio/src/gacto2.str
Structure processing report: /usr/local/timelink/clio/src/gacto2.srpt
Structure in JSON: /usr/lo

### Get the import report for a file

In case there are errors in the import phase check the import report.

In [9]:
rpt = tlnb.get_import_rpt(imported_files_df, rows=[0])
print(rpt)

b1685.cli
No errors
No warnings


